In [19]:
# 1. Load environment variables
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
# Make sure your .env contains:
PPLX_API_KEY= "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"
TAVILY_API_KEY= "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"

In [17]:
# 2. Set up Tavily Search tool
from langchain_tavily import TavilySearch

search = TavilySearch(
    max_results=3,  # Or as needed
    topic="general"
)

tools = [search]



In [20]:
# 3. Set up Perplexity LLM using ChatOpenAI with Perplexity's base_url
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model="llama-3.1-sonar-small-128k-online",  # Or "sonar-pro"
    temperature=0.7,
    api_key=os.environ.get("PPLX_API_KEY"),  # Your Perplexity API key
    base_url="https://api.perplexity.ai"      # Point to Perplexity's API
)

# 4. Create LangGraph agent
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

# 5. Run a query using LangGraph agent
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({
    "messages": [HumanMessage(content="Tell me about Bill Gates and his contributions to technology and philanthropy.")]
})
print(response)

# 6. (Optional) Access the response messages
print(response["messages"])


{'messages': [HumanMessage(content='Tell me about Bill Gates and his contributions to technology and philanthropy.', additional_kwargs={}, response_metadata={}, id='a0b12fcd-1c0b-4ce5-bd77-5d9278872bb2'), AIMessage(content='Bill Gates is a renowned software developer and philanthropist who co-founded Microsoft, one of the most influential technology companies of the 20th and 21st centuries. His contributions to technology and philanthropy are multifaceted and enduring.\n\n### Contributions to Technology\n\n1. **Personal Computing Revolution**:\n   - **Microsoft Co-Founder**: Gates co-founded Microsoft in 1975 with Paul Allen. He played a pivotal role in revolutionizing personal computing by developing software that made computers accessible to the general public[2][5].\n   - **MS-DOS and Windows**: He was instrumental in the development of MS-DOS and later Windows, which became the dominant operating systems for personal computers[5].\n   - **Microsoft Office**: Gates also spearheaded 

In [22]:
# 7. Save agent response with content and source links

import datetime
import re

def save_agent_response(response, filename="agent_logs.txt"):
    """Save agent response with content and source links to a text file"""
    
    # Extract messages and content
    messages = response.get("messages", [])
    content = "\n".join([msg.content for msg in messages if hasattr(msg, 'content')])
    
    # Find all URLs in the response
    urls = re.findall(r'https?://[^\s\>\<\"]+', str(response))
    
    # Prepare formatted text
    log_entry = f"""
{'='*60}
{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*60}
RESPONSE:
{content}

SOURCE LINKS:
{'\n'.join(['- '+url for url in urls]) if urls else 'No links found'}
{'='*60}\n\n
"""
    
    # Append to file
    with open(filename, "a", encoding="utf-8") as file:
        file.write(log_entry)
    print(f"📄 Appended response to {filename}")

# Modified agent execution with auto-saving
def run_agent(query):
    response = agent_executor.invoke({
        "messages": [HumanMessage(content=query)]
    })
    save_agent_response(response)
    return response

# Usage example:
response = run_agent("Tell me about Bill Gates and his contributions to technology and philanthropy.")


📄 Appended response to agent_logs.txt


In [23]:
# 8. Example of running a query and saving the response Using memory 

# 9. Environment Setup
import os
from dotenv import load_dotenv
load_dotenv()

# 10. Configure Perplexity LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="llama-3.1-sonar-small-128k-online",
    api_key=os.getenv("PPLX_API_KEY"),
    base_url="https://api.perplexity.ai"
)

# 11. Tavily Search Tool Setup
from langchain_tavily import TavilySearch
search = TavilySearch(
    max_results=3,
    api_key=os.getenv("TAVILY_API_KEY")
)
tools = [search]

# 12. Memory Configuration
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

memory = MemorySaver()
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# 13. Conversation Configuration
config = {"configurable": {"thread_id": "001"}}

# 14. First Query Execution - CORRECTED FORMAT
print("=== FIRST QUESTION ===")
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Which team won the TATA IPL 2011?")]},
    config
):
    print(chunk)
    print("---------")

# 15. Follow-up Query Using Memory - CORRECTED FORMAT
print("\n=== FOLLOW-UP QUESTION ===")
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who was the captain of the winning team?")]},
    config
):
    print(chunk)
    print("---------")

    


=== FIRST QUESTION ===
{'agent': {'messages': [AIMessage(content='The team that won the TATA IPL 2011 was the **Chennai Super Kings**. They defeated the **Royal Challengers Bangalore** by 58 runs in the final on May 28, 2011, at the M. A. Chidambaram Stadium in Chennai[1][3]. This was their second consecutive IPL title win[3].', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 11, 'total_tokens': 86, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'search_context_size': 'low'}, 'model_name': 'llama-3.1-sonar-small-128k-online', 'system_fingerprint': None, 'id': '0d45ec96-e0b1-4dbf-bc52-b04441267676', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--857a75ff-904a-4602-b480-369996f96c64-0', usage_metadata={'input_tokens': 11, 'output_tokens': 75, 'total_tokens': 86, 'input_token_details': {}, 'output_token_details': {}})]}}
---------

=== FOLLOW-UP QUESTION ===
{'agent': {'message